In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[6000:7000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_7.json","w") as f:
      f.write(data)

with open("film_data_7.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Robert De Niro+_+Jean Reno+_+Natascha McElh...,"$55,000,000\n",+_+UK+_+France+_+USA,+_+John Frankenheimer,+_+Action+_+Adventure+_+Crime+_+Thriller,"$41,616,262\n",+_+ronin+_+case+_+suitcase+_+russian+_+ex kgb,+_+English+_+French+_+Russian,R,7.3,1998-09-25,122.0,Ronin (1998),"154,595",+_+J.D. Zeik+_+J.D. Zeik,1998
1,+_+Bill Paxton+_+Bridget Fonda+_+Billy Bob Tho...,"$17,000,000\n",+_+UK+_+Germany+_+France+_+USA+_+Japan,+_+Sam Raimi,+_+Crime+_+Drama+_+Thriller,"$16,311,763,",+_+bag of money+_+snow+_+plane wreck+_+screenp...,+_+English,R,7.5,1999-01-22,121.0,A Simple Plan (1998),"54,803",+_+Scott B. Smith+_+Scott B. Smith,1998
2,+_+Ben Stiller+_+Maria Bello+_+Jay Paulson+_+S...,None,+_+USA,+_+David Veloz,+_+Biography+_+Drama+_+Romance,"$1,166,199,",+_+writer+_+habit+_+flashback+_+detox+_+heroin,+_+English,R,6.3,1999-04-23,88.0,Permanent Midnight (1998),"6,326",+_+Jerry Stahl+_+David Veloz,1998
3,+_+Casey Affleck+_+Brendan Sexton III+_+Kate H...,"$2,000,000\n",+_+USA,+_+Morgan J. Freeman,+_+Drama,"$99,147,",+_+picnic table+_+deck chair+_+teenage girl+_+...,+_+English,R,6.2,1999-06-18,90.0,Desert Blue (1998),"1,993",+_+Morgan J. Freeman,1998
4,+_+Billy Zane+_+Tippi Hedren+_+Ron Perlman+_+M...,None,+_+USA,+_+Aris Iliopulos,+_+Comedy+_+Crime+_+Thriller,None,+_+neo noir+_+cult movie cast+_+money+_+expres...,+_+English+_+Spanish,None,6.1,1999-09-10,90.0,I Woke Up Early the Day I Died (1998),568,+_+Edward D. Wood Jr.,1998
5,+_+Harvey Keitel+_+Johnathon Schaech+_+Bridget...,"$10,000,000\n",+_+USA,+_+David Winkler,+_+Biography+_+Drama,None,+_+graceland+_+elvis presley+_+impersonator+_+...,+_+English,PG-13,6.5,1999-07-28,97.0,Finding Graceland (1998),"1,721",+_+Jason Horwitch+_+David Winkler,1998
6,+_+Sean Astin+_+Emily Hampshire+_+Kevin McDona...,None,+_+Canada+_+USA,+_+Jerry Ciccoritti,+_+Comedy+_+Fantasy+_+Romance,None,+_+writer+_+canada+_+italian woman+_+fiance+_+...,+_+English+_+Italian,None,6.2,1998-09-12,NaN,Boy Meets Girl (1998),273,"+_+Doug Bagot,+_+Timothy Lee",1998
7,+_+Steve Taylor+_+Marvin Schoenberg+_+Susy Wri...,"$15,000\n",+_+USA,+_+Steve Taylor,+_+Comedy,None,+_+geek+_+hook up+_+revenge+_+pick up line+_+l...,+_+English,NOT RATED,4.2,2000-01-15,88.0,Social Intercourse (1998),122,"+_+Roger Khristian Jones,+_+Steve Taylor",1998
8,+_+Kris Kristofferson+_+Barbara Hershey+_+Leel...,None,+_+UK+_+France+_+USA,+_+James Ivory,+_+Drama,"$1,759,596,",+_+author+_+adopted son+_+1970s+_+teenage boy+...,+_+English+_+French,R,6.9,1998-09-18,127.0,A Soldier's Daughter Never Cries (1998),"2,132",+_+Kaylie Jones+_+James Ivory,1998
9,+_+Minnie Driver+_+Nigel Hawthorne+_+Rufus Sew...,None,+_+USA,+_+John Huddles,+_+Drama,None,+_+independent film,+_+English,PG,6.5,2001-05-02,106.0,At Sachem Farm (1998),514,+_+John Huddles,1998
